In [1]:
# https://www.mathworks.com/help/econ/empiricalblm.html#bvjyd4x-1
# bayesian linear regression mo
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
import pandas as pd
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [5]:
df = pd.read_csv('data_20240529.csv', dtype=np.float128)
df.shape

(2540, 19)

In [ ]:
# estimate the model
X = df.drop('energy', axis=1)
y = df['energy']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
del X, y

In [ ]:
import pymc3 as pm

# Step 1: Define the model structure
with pm.Model() as model:
    # Define prior distributions for the parameters
    alpha = pm.Normal('alpha', mu=0, sd=10)
    beta = pm.Normal('beta', mu=0, sd=10, shape=len(X_train.columns))

    # Linear regression model
    mu = alpha + pm.math.dot(beta, X_train.T)

    # Step 2: Define the likelihood function
    sigma = pm.HalfNormal('sigma', sd=1)
    likelihood = pm.Normal('y', mu=mu, sd=sigma, observed=y_train)

    # Step 4: Sample from the posterior distribution, this is simulate the posterior distribution, 
    # which is the distribution of the parameters given the data, using the No-U-Turn Sampler (NUTS) algorithm, 
    # a variant of Hamiltonian Monte Carlo (HMC), to generate samples from the posterior distribution.
    trace = pm.sample(2000, tune=1000, cores=1)

    # Step 5: Analyze the posterior distribution
    pm.traceplot(trace)
    plt.show()

    # Step 6: Make predictions, use X_test, y_test
    
    y_pred = pm.sample_posterior_predictive(trace, samples=2000, model=model)
    y_pred = y_pred['y']

# Access the samples from the posterior distribution
parameter1_samples = trace['alpha']
parameter2_samples = trace['beta']

# Distribution summary statistics of standard Bayesian linear regression model
print(pm.summary(trace))

In [ ]:
# Forecast responses of Bayesian linear regression model